# Using ZhipuAI GLM-4 though OpenAI SDK and Ecosystem

**This tutorial is available in English and is attached below the Chinese explanation**

本节代码将带领大家使用`OpenAI`的 SDK 和支持的框架调用 `GLM-4` 模型
你将在本章节学习到如何使用 Langchain 的 `ChatOpenAI` 类来调用 ZhipuAI 的 GLM-4 模型，以及如何使用 `OpenAI SDK` 类来实现调用GLM-4模型。

This notebook section will walk you through calling `GLM-4` models using the `OpenAI` SDK and supporting frameworks.
In this section you will learn how to call ZhipuAI's GLM-4 models using Langchain's `ChatOpenAI` class, and how to use the `OpenAI SDK` class to implement calling GLM-4 models.


## 1. Use OpenAI SDK

首先，需要导入 `OpenAI` SDK，同时，修改 `OpenAI` 的 `base_url` 为 `https://open.bigmodel.cn/api/paas/v4/` 
以便调用 `GLM-4` 模型。完成更换后，仅需填写 Key 就能使用 `GLM-4` 模型。

First, you need to import the `OpenAI` SDK and change the `OpenAI` `base_url` to `https://open.bigmodel.cn/api/paas/v4/` in order to call the `GLM-4` model. 
in order to call the `GLM-4` model. Once you have done this, you can use the `GLM-4` model by simply filling in the Key.
``python

In [3]:
from openai import OpenAI
import os
client = OpenAI(
    api_key=os.getenv("ZHIPUAI_API_KEY"),
    base_url="https://open.bigmodel.cn/api/paas/v4/")

## 2. Simple use of OpenAI SDK to call GLM-4

我将带领大家完成最基础的对话调用。
请注意：
- `temperature` 参数的区间为 (0,1)。
- `do_sample = False (temperature = 0)` 在 OpenAI 调用中并不适用。
-  异步操作不适用。

其他例子(例如工具调用)，也均可以按照OpenAI的方式进行调用。


I will lead everyone to complete the most basic dialogue calls.
Please note:

- The interval of the `temperature` parameter is (0,1).
- `do_sample=False (temperature=0)` is not applicable in OpenAI calls.
- Asynchronous operations are not applicable.

Other examples (such as tool calls) can also be called in the same way as OpenAI.

In [4]:
response = client.chat.completions.create(
    model="glm-4",
    messages=[
        {
            "role": "user",
            "content": "tell me a joke"
        }
    ],
    top_p=0.7,
    temperature=0.9,
    stream=False,
    max_tokens=2000,
)
response

ChatCompletion(id='8602219229372288758', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure, here's a light-hearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!", role='assistant', function_call=None, tool_calls=None))], created=1714098003, model='glm-4', object=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=26, prompt_tokens=9, total_tokens=35), request_id='8602219229372288758')

## 3. Using Langchain's ChatOpenAI class

Langchain 的 `ChatOpenAI` 类是对 `OpenAI` SDK 的封装，可以更方便调用。这里展示了如何使用 `ChatOpenAI` 类来调用 `GLM-4` 模型。

Langchain's `ChatOpenAI` class is a wrapper around the `OpenAI` SDK, making it easier to call. Here we show how to use the `ChatOpenAI` class to call the `GLM-4` model.

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.01,
    model="glm-4",
    openai_api_key=os.getenv("ZHIPUAI_API_KEY"),
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)
conversation.invoke({"question": "tell me a joke"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: tell me a joke

> Finished chain.


{'question': 'tell me a joke',
 'chat_history': [HumanMessage(content='tell me a joke'),
  AIMessage(content="Sure! Here's a light-hearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything! 😄")],
 'text': "Sure! Here's a light-hearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything! 😄"}

## 4. Calling GLM-4 with LangChain AgentExecutor
同时，GLM-4也能很好的接入到 LangChain 的 AgentExecutor 中，这里展示了如何使用 `AgentExecutor` 来调用 `GLM-4` 模型。

GLM-4 also plugs nicely into LangChain's AgentExecutor, which shows how to call a `GLM-4` model using `AgentExecutor`.

In [8]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent,create_tool_calling_agent,initialize_agent
from langchain_community.tools.tavily_search import TavilySearchResults

# os.environ["TAVILY_API_KEY"] = "tvly-xxx"
tools = [TavilySearchResults(max_results=2)]
prompt = hub.pull("hwchase17/react")
# Choose the LLM to use
agent = create_react_agent(llm, tools, prompt)
# agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "create_tool_calling_agent 和 create_react_agent,create_json_chat_agent 的区别?"})



> Entering new AgentExecutor chain...
These seem to be specific functions or agents related to a particular software framework or system. I am not familiar with these specific functions, but I can try to search for information about them. 

Action: tavily_search_results_json
Action Input: differences between create_tool_calling_agent and create_react_agent,create_json_chat_agent
Observation[{'url': 'https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html', 'content': 'A Runnable sequence representing an agent. It takes as input all the same input variables as the prompt passed in does. It returns as output either an AgentAction or AgentFinish. Examples: from langchain import hub from langchain_community.llms import OpenAI from langchain.agents import AgentExecutor, create_react_agent prompt = hub.pull ...'}, {'url': 'https://medium.com/@emil.azadian/using-agents-as-tools-in-langchain-1758878792b2', 'content': 'The docs suggest using the 

{'input': 'create_tool_calling_agent 和 create_react_agent,create_json_chat_agent 的区别?',
 'output': 'The differences between create_tool_calling_agent, create_react_agent, and create_json_chat_agent are not clearly mentioned in the available search results. It is recommended to refer to the LangChain documentation or explore reliable sources for more accurate information.'}